In [42]:
# Imports


# Geometry
from h3 import h3
from shapely.geometry import Point
from shapely.geometry import Polygon
from geopy.distance import distance
from geopy.distance import geodesic
from shapely.wkt import loads

# Visualization
import matplotlib
import matplotlib.pyplot as plt
import folium
import contextily as ctx

# Projections
import pyproj

# Data handling
import geopandas as gpd
import pandas as pd
import numpy as np
import ast
from pandas.io.json import json_normalize

# Network
import requests
import time
import osmnx as ox
import sklearn
import networkx as nx


POIS

In [43]:
POIS = pd.read_csv('POIS_final.csv', sep='|')

In [58]:
# Create a map
mp = folium.Map(location=[POIS['lat'].mean(), POIS['lon'].mean()], zoom_start=13)

# Plot the POIs
for i, row in POIS.iterrows():
    folium.CircleMarker(location=[row['lat'], row['lon']], radius=5, color='blue', fill=True, fill_color='red').add_to(mp)

# Plot the shapefile
#folium.GeoJson(toctiuco_shp).add_to(m)

# Show the map
mp


H3 La Bota y Toctiuco

In [59]:
#Read and format Shape Files
labota_shp = gpd.read_file("C:/Users/diana/Documents/healthy_cities/healthy_cities/Healthy_cities/LaBotayToctiuco/CENSOLABOTA/CENSO.shp")
labota_shp = labota_shp.to_crs('epsg:4326') # lat lon format

#Read and format Shape Files
toctiuco_shp = gpd.read_file("C:/Users/diana/Documents/healthy_cities/healthy_cities/Healthy_cities/LaBotayToctiuco/CENSOTOCTIUCO/CENSO.shp")
toctiuco_shp = toctiuco_shp.to_crs('epsg:4326')

# Extract the bounding box of the labota_shp
bbox1 = labota_shp.total_bounds
polygon1 = Polygon([(bbox1[0], bbox1[1]), (bbox1[0], bbox1[3]), (bbox1[2], bbox1[3]), (bbox1[2], bbox1[1])])
#Get the centroid of the bounding box
centroid1 = polygon1.centroid

# Extract the bounding box of the labota_shp
bbox2 = toctiuco_shp.total_bounds
polygon2 = Polygon([(bbox2[0], bbox2[1]), (bbox2[0], bbox2[3]), (bbox2[2], bbox2[3]), (bbox2[2], bbox2[1])])
#Get the centroid of the bounding box
centroid2 = polygon2.centroid

In [60]:
polygon1 = {
'type': 'Polygon',
'coordinates': [
[
[bbox1[1], bbox1[0]],
[bbox1[1], bbox1[2]],
[bbox1[3], bbox1[2]],
[bbox1[3], bbox1[0]],
[bbox1[1], bbox1[0]]
]
]
}

resolution = 10
hexagons1 = h3.polyfill(polygon1, resolution, geo_json_conformant=False)
hexagons1_list = list(hexagons1)

polygon2 = {
'type': 'Polygon',
'coordinates': [
[
[bbox2[1], bbox2[0]],
[bbox2[1], bbox2[2]],
[bbox2[3], bbox2[2]],
[bbox2[3], bbox2[0]],
[bbox2[1], bbox2[0]]
]
]
}

resolution = 10
hexagons2 = h3.polyfill(polygon2, resolution, geo_json_conformant=False)
hexagons2_list = list(hexagons2)



polygonise = lambda hex_id: Polygon(
                                h3.h3_to_geo_boundary(
                                    hex_id, geo_json=True)
                                    )

all_polys1 = gpd.GeoSeries(list(map(polygonise, hexagons1)), \
                                      index=hexagons1, \
                                      crs="EPSG:4326" \
                                     )

all_polys2 = gpd.GeoSeries(list(map(polygonise, hexagons2)), \
                                      index=hexagons2, \
                                      crs="EPSG:4326" \
                                     )



In [61]:
# Create a folium map centered on LaBota
lat1, lon1 = centroid1.y, centroid1.x
m1 = folium.Map(location=[lat1, lon1], zoom_start=10 )
folium.GeoJson(all_polys1).add_to(m1)


display(m1)

In [62]:
# Create a folium map centered on LaBota
lat2, lon2 = centroid2.y, centroid2.x
m2 = folium.Map(location=[lat2, lon2], zoom_start=10 )
folium.GeoJson(all_polys2).add_to(m2)


display(m2)

Cruce de capas

In [68]:
# Creación del mapa
mapa = folium.Map(location=[POIS['lat'].mean(), POIS['lon'].mean()], zoom_start=10)
# Añadir capas
m1_final = folium.FeatureGroup(name='H3_labota')

m1_final = folium.GeoJson(all_polys1)

m2_final = folium.FeatureGroup(name='H3_toctiuco')

m2_final = folium.GeoJson(all_polys2)

mp_final = folium.FeatureGroup(name='POIS')

for i, row in POIS.iterrows():
    folium.CircleMarker(location=[row['lat'], row['lon']], radius=5, color='blue', fill=True, fill_color='red').add_to(mp_final)


# Agregar FeatureGroups al mapa
mapa.add_child(m1_final)
mapa.add_child(m2_final)
mapa.add_child(mp_final)
folium.LayerControl().add_to(mapa)

In [69]:
mapa